In [182]:
import pandas as pd
from keras.layers import Input, Embedding, LSTM, Dense, Dot, Softmax, Concatenate, BatchNormalization, Attention
from keras.models import Model
import tensorflow as tf
import numpy as np
from keras.utils import pad_sequences
from datetime import datetime

In [183]:
basepath = '../../datasets/preprocessed_datasets/gabor/'
product_data = pd.read_pickle(basepath +'orders_and_products_by_customer.pkl')

In [184]:
product_data = product_data.sample(n=1000)
# convert dates to ordinal
product_data['order_dates'] = product_data['order_dates'].apply(lambda x: [ [datetime.fromtimestamp(date).date().toordinal() for date in dateList] for dateList in x] )
product_data

,customer_id,order_ids,product_ids,product_names,amounts,total_prices,order_dates
181057,7503484.0,[7503482],[[3555072]],[[Chelsea Boot Rauleder blau]],[[1]],[[115.0]],[[738116]]
338149,10735271.0,[10735314],"[[8501588, 7789353]]","[[Sneaker low Materialmix Leder multicolour, S...","[[1, 1]]","[[69.0, 89.0]]","[[738560, 738560]]"
58136,4606091.0,[4606088],[[3788155]],[[Sneaker high Rauleder schwarz]],[[1]],[[116.97]],[[737736]]
289701,9827665.0,[9827663],[[8499597]],[[Chelsea Boot Rauleder grau]],[[1]],[[99.95]],[[738442]]
278205,9641298.0,[9641296],[[8488231]],[[Sneaker low Rauleder schwarz]],[[1]],[[140.0]],[[738423]]
...,...,...,...,...,...,...,...
219640,8446444.0,[8446441],[[7591358]],[[Spangenpumps Rauleder beige]],[[1]],[[99.95]],[[738269]]
223017,8546041.0,[8546037],"[[7592779, 7592229]]","[[Slingpumps Glattleder weiß, Slingpumps Raule...","[[1, 1]]","[[99.95, 99.95]]","[[738278, 738278]]"
59799,4646119.0,[4646117],[[3844418]],[[Plateau Stiefel Glattleder schwarz]],[[1]],[[175.46]],[[737741]]
220393,8469823.0,[8469821],[[3847667]],[[Spangenpumps Rauleder blau]],[[1]],[[99.95]],[[738271]]


In [185]:
def bringToSameSizeOneDim(column):
    max_length = max(column.apply(len))
    return column.apply(lambda x: x + [0] * (max_length - len(x)))


def bringToSameSizeTwoDimAndFlatten(column):
    max_length = max(column.apply(lambda x: max(len(sublist) for sublist in x)))
    new_column = column.apply(lambda x: [sublist + [0] * (max_length - len(sublist)) for sublist in x])
    new_column = new_column.apply(lambda x: x + [[0] * max_length] * (max_length - len(x)))
    return new_column.apply(lambda x: [item for sublist in x for item in sublist])

product_data.order_ids = bringToSameSizeOneDim(product_data.order_ids)
product_data.product_ids = bringToSameSizeTwoDimAndFlatten(product_data.product_ids)
product_data.amounts = bringToSameSizeTwoDimAndFlatten(product_data.amounts)
product_data.total_prices = bringToSameSizeTwoDimAndFlatten(product_data.total_prices)
product_data.order_dates = bringToSameSizeTwoDimAndFlatten(product_data.order_dates)


In [186]:
def convertToSequence(column):
    list = column.tolist()
    max_length = max(len(x) for x in list)
    return pad_sequences(list, maxlen=max_length, padding='post', value=0), max_length

products_inputs, max_length_products = convertToSequence(product_data.product_ids)
dates_inputs, max_length_dates = convertToSequence(product_data.order_dates)
target_data, _ = convertToSequence(product_data.amounts)
target_data = np.expand_dims(target_data, axis=-1)
num_targets = target_data.shape[1]

In [187]:
display(max_length_dates, dates_inputs.shape[1], target_data.shape)
display(target_data)
display(dates_inputs)
display(products_inputs)
# 1602099712

80

80

(1000, 80, 1)

array([[[1],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]],

       [[1],
        [1],
        [0],
        ...,
        [0],
        [0],
        [0]],

       [[1],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]],

       ...,

       [[1],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]],

       [[1],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]],

       [[1],
        [1],
        [0],
        ...,
        [0],
        [0],
        [0]]], dtype=int32)

array([[738116,      0,      0, ...,      0,      0,      0],
       [738560, 738560,      0, ...,      0,      0,      0],
       [737736,      0,      0, ...,      0,      0,      0],
       ...,
       [737741,      0,      0, ...,      0,      0,      0],
       [738271,      0,      0, ...,      0,      0,      0],
       [738052, 738052,      0, ...,      0,      0,      0]], dtype=int32)

array([[3555072,       0,       0, ...,       0,       0,       0],
       [8501588, 7789353,       0, ...,       0,       0,       0],
       [3788155,       0,       0, ...,       0,       0,       0],
       ...,
       [3844418,       0,       0, ...,       0,       0,       0],
       [3847667,       0,       0, ...,       0,       0,       0],
       [3845626, 3481847,       0, ...,       0,       0,       0]],
      dtype=int32)

In [188]:
# Define the model architecture
# num_orders = sum(len(order) for order in product_data.order_ids)
# num_products =  sum(len(order) for order in product_data.product_ids)
# num_units = len([amount for order_amounts in product_data.amounts for amount in order_amounts])
# num_prices = len([amount for order_amounts in product_data.total_prices for amount in order_amounts])
# num_dates = len([amount for order_amounts in product_data.order_dates for amount in order_amounts])
#
# print(num_orders, num_products, num_units, num_prices, num_dates)

In [189]:
num_dates = len(np.unique(product_data.order_dates))
num_products = len(np.unique(product_data.product_ids))
max_index = max(np.max(dates_inputs), np.max(products_inputs))
vocab_size = max_index + 1
max_date = np.max(np.unique(product_data.order_dates.apply(np.unique).apply(max))) + 1
max_product_ids = np.max(np.unique(product_data.product_ids.apply(np.unique).apply(max))) + 1

In [190]:
from keras.layers import Reshape
date_embedding_dim = 16
product_embedding_dim = 16
date_inputs = Input(shape=(max_length_dates,))
product_inputs = Input(shape=(max_length_products,))

print(dates_inputs.shape)
print(products_inputs.shape)
print(target_data.shape)
print(num_dates, num_products, max_date, max_product_ids)
encoding_padding_mask = tf.math.logical_not(tf.math.equal(product_inputs, 0))

# Embedding layers
#old version
#date_emb = Embedding(num_dates, 16, input_length=max_length_dates)(date_inputs)
#product_emb = Embedding(num_products, 16, input_length=max_length_products)(product_inputs)

#improved version loads to long
#date_emb = Embedding(max_date, date_embedding_dim)(date_inputs)
#product_emb = Embedding(max_product_ids, product_embedding_dim)(product_inputs)

date_emb = Embedding(max_date, date_embedding_dim, input_length=max_length_dates)(date_inputs)
product_emb = Embedding(max_product_ids, product_embedding_dim, input_length=max_length_products)(product_inputs)


concat_embedding_input = Concatenate(
    axis=-1, name='concat_embedding_input')([date_emb, product_emb])

batchnorm_inputs = BatchNormalization(
    name='batchnorm_inputs')(concat_embedding_input)
# LSTM layer
lstm = LSTM(64, return_sequences=True)(concat_embedding_input)

lstm = BatchNormalization(name='batchnorm_lstm')(lstm)

att = Attention(use_scale=False,
                name='attention')(inputs=[lstm, lstm],
                                  mask=[encoding_padding_mask,
                                        encoding_padding_mask])

output = Dense(num_targets)(att)
# Define the model
model = Model(inputs=[date_inputs, product_inputs], outputs=output)
# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse', 'mae', 'accuracy'])
model.summary()

(1000, 80)
(1000, 80)
(1000, 80, 1)
785 990 738603 10787222
Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_23 (InputLayer)          [(None, 80)]         0           []                               
                                                                                                  
 input_24 (InputLayer)          [(None, 80)]         0           []                               
                                                                                                  
 embedding_21 (Embedding)       (None, 80, 16)       11817648    ['input_23[0][0]']               
                                                                                                  
 embedding_22 (Embedding)       (None, 80, 16)       172595552   ['input_24[0][0]']               
                               

In [191]:
history = model.fit([dates_inputs, products_inputs], target_data, epochs=15, batch_size=32)  # Adjust epochs and batch size as needed

Epoch 1/15


2023-06-04 14:30:26.233255: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:690] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" model: "0" num_cores: 10 environment { key: "cpu_instruction_set" value: "ARM NEON" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 16384 l2_cache_size: 524288 l3_cache_size: 524288 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


32/32 [==============================] - 16s 457ms/step - loss: 0.3518 - mse: 0.3455 - mae: 0.4325 - accuracy: 0.0000e+00
Epoch 2/15
32/32 [==============================] - 14s 450ms/step - loss: 0.0150 - mse: 0.0153 - mae: 0.0908 - accuracy: 0.0403
Epoch 3/15
32/32 [==============================] - 14s 449ms/step - loss: 0.0081 - mse: 0.0083 - mae: 0.0625 - accuracy: 0.0132
Epoch 4/15
32/32 [==============================] - 15s 454ms/step - loss: 0.0066 - mse: 0.0064 - mae: 0.0523 - accuracy: 0.0160
Epoch 5/15
32/32 [==============================] - 14s 450ms/step - loss: 0.0053 - mse: 0.0053 - mae: 0.0456 - accuracy: 0.0143
Epoch 6/15
32/32 [==============================] - 14s 451ms/step - loss: 0.0042 - mse: 0.0045 - mae: 0.0404 - accuracy: 0.0160
Epoch 7/15
32/32 [==============================] - 14s 452ms/step - loss: 0.0041 - mse: 0.0040 - mae: 0.0361 - accuracy: 0.0077
Epoch 8/15
32/32 [==============================] - 14s 449ms/step - loss: 0.0035 - mse: 0.0036 - mae: 0

In [192]:
display(history.history)

{'loss': [0.351816862821579,
  0.01502153929322958,
  0.008095471188426018,
  0.0065895263105630875,
  0.005281096789985895,
  0.0041811238043010235,
  0.004101834259927273,
  0.0034972266294062138,
  0.0032134170178323984,
  0.003178244223818183,
  0.002908071270212531,
  0.0026728189550340176,
  0.0026786071248352528,
  0.0021967978682368994,
  0.002228684024885297],
 'mse': [0.3455391824245453,
  0.015285098925232887,
  0.008267495781183243,
  0.006376548204571009,
  0.005301357246935368,
  0.004527673590928316,
  0.003989415243268013,
  0.003555122995749116,
  0.0032854583114385605,
  0.0031015793792903423,
  0.0028221372049301863,
  0.0026780846528708935,
  0.0024356618523597717,
  0.0022517626639455557,
  0.0022837535943835974],
 'mae': [0.43251633644104004,
  0.09075537323951721,
  0.06253445893526077,
  0.05233281850814819,
  0.04561420902609825,
  0.04038911685347557,
  0.03614810109138489,
  0.03288885951042175,
  0.030425898730754852,
  0.029379213228821754,
  0.027226543053

In [193]:
# Evaluate the model
loss, mse, mae, acc = model.evaluate([dates_inputs, products_inputs], target_data)
print("Test loss: {:.4f}, Test MSE: {:.4f}".format(loss, mse))
print("Test MAE: {:.4f}, Test Accuracy: {:.4f}".format(mae, acc))

15/32 [=============>................] - ETA: 0s - loss: 0.4471 - mse: 0.4491 - mae: 0.6671 - accuracy: 0.0000e+00

2023-06-04 14:34:02.990522: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:690] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" model: "0" num_cores: 10 environment { key: "cpu_instruction_set" value: "ARM NEON" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 16384 l2_cache_size: 524288 l3_cache_size: 524288 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


32/32 [==============================] - 0s 8ms/step - loss: 0.4454 - mse: 0.4508 - mae: 0.6680 - accuracy: 0.0000e+00
Test loss: 0.4454, Test MSE: 0.4508
Test MAE: 0.6680, Test Accuracy: 0.0000
